In [1]:
# pip install kafka-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 10.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
from os.path import abspath
from kafka import KafkaConsumer
from json import loads
from time import sleep
import pyspark
from pyspark.sql import SparkSession

In [4]:
warehouse_location = abspath('spark-warehouse')
scSpark = SparkSession.builder \
    .appName("Spark Hive") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .enableHiveSupport() \
    .getOrCreate()

In [4]:
## MEMBUAT TABLE
scSpark.sql("CREATE TABLE IF NOT EXISTS laporan_cuaca(kota string, cuaca string,\
            cuaca_deskripsi string, temperatur string, temperatur_min string, temperatur_max string,\
            date string, waktu string, created string) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' ;")

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used
22/08/09 14:38:10 WARN org.apache.hadoop.hive.ql.session.SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


DataFrame[]

In [5]:
scSpark.sql("SELECT * FROM laporan_cuaca")

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used


DataFrame[kota: string, cuaca: string, cuaca_deskripsi: string, temperatur: string, temperatur_min: string, temperatur_max: string, date: string, waktu: string, created: string]

In [6]:
data_input = {
    "kota": "Bali",
    "cuaca": "Clouds",
    "cuaca_deskripsi": "overcast clouds",
    "temperatur": 299.39,
    "temperatur_min": 299.39,
    "temperatur_max": 299.69,
    "date": "2022-08-09",
    "waktu": "06:31:41",
    "created": "2022-08-09T06:31:41.124657"
}

In [7]:
insert_data = "('"+\
    data_input['kota']+"','"+\
    data_input['cuaca']+"','"+\
    data_input['cuaca_deskripsi']+"','"+\
    str(data_input['temperatur'])+"','"+\
    str(data_input['temperatur_min'])+"','"+\
    str(data_input['temperatur_max'])+"','"+\
    str(data_input['date'])+"','"+\
    str(data_input['waktu'])+"','"+\
    str(data_input['created'])+\
    "')"
print(insert_data)

('Bali','Clouds','overcast clouds','299.39','299.39','299.69','2022-08-09','06:31:41','2022-08-09T06:31:41.124657')


In [8]:
sql_command = "INSERT INTO laporan_cuaca values"+insert_data
print(sql_command)

INSERT INTO laporan_cuaca values('Bali','Clouds','overcast clouds','299.39','299.39','299.69','2022-08-09','06:31:41','2022-08-09T06:31:41.124657')


In [9]:
scSpark.sql("INSERT INTO laporan_cuaca values('Bali','Clouds','overcast clouds','299.39','299.39','299.69','2022-08-09','06:31:41','2022-08-09T06:31:41.124657')")

DataFrame[]

In [10]:
scSpark.sql(sql_command)

DataFrame[]

In [4]:
scSpark.sql("SELECT * FROM laporan_cuaca").show()

22/08/12 07:41:03 WARN org.apache.hadoop.util.concurrent.ExecutorHelper: Thread (Thread[GetFileInfo #1,5,main]) interrupted: 
java.lang.InterruptedException
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:510)
	at com.google.common.util.concurrent.FluentFuture$TrustedFuture.get(FluentFuture.java:88)
	at org.apache.hadoop.util.concurrent.ExecutorHelper.logThrowableFromAfterExecute(ExecutorHelper.java:48)
	at org.apache.hadoop.util.concurrent.HadoopThreadPoolExecutor.afterExecute(HadoopThreadPoolExecutor.java:90)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1157)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:750)


+-----------+------+----------------+----------+--------------+--------------+----------+--------+--------------------+
|       kota| cuaca| cuaca_deskripsi|temperatur|temperatur_min|temperatur_max|      date|   waktu|             created|
+-----------+------+----------------+----------+--------------+--------------+----------+--------+--------------------+
|    Bandung|Clouds|   broken clouds|    294.28|        294.28|        294.28|2022-08-09|14:58:13| 2022-08-09T14:58:13|
|      Medan|  Haze|            haze|    299.83|        299.83|        300.17|2022-08-09|14:58:12| 2022-08-09T14:58:12|
|     Lombok|Clouds| overcast clouds|    294.42|        294.42|        294.42|2022-08-09|13:49:15| 2022-08-09T13:49:15|
|     Lombok|Clouds|   broken clouds|    294.07|        294.07|        294.07|2022-08-09|14:58:12| 2022-08-09T14:58:12|
|Labuan Bajo|Clouds|      few clouds|    297.42|        297.42|        297.42|2022-08-09|14:58:12| 2022-08-09T14:58:12|
|       Bali|Clouds| overcast clouds|   

In [ ]:
consumer = KafkaConsumer(
    'laporan_cuaca',
    bootstrap_servers=['34.87.9.155:9092'],
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='rifat-dataproc-suhu',
    value_deserializer=lambda x: loads(x.decode('utf-8'))
)
for event in consumer:
    event_data = event.value
    data_input = event_data
    
    insert_data = "('"+\
        data_input['kota']+"','"+\
        data_input['cuaca']+"','"+\
        data_input['cuaca_deskripsi']+"','"+\
        str(data_input['temperatur'])+"','"+\
        str(data_input['temperatur_min'])+"','"+\
        str(data_input['temperatur_max'])+"','"+\
        str(data_input['date'])+"','"+\
        str(data_input['waktu'])+"','"+\
        str(data_input['created'])+\
        "')"
    sql_command = "INSERT INTO laporan_cuaca values"+insert_data
    scSpark.sql(sql_command)
    print(event_data)
    sleep(2)

22/08/12 07:41:34 WARN org.apache.hadoop.hive.ql.session.SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


{'kota': 'Bali', 'cuaca': 'Clouds', 'cuaca_deskripsi': 'broken clouds', 'temperatur': 300.35, 'temperatur_min': 300.35, 'temperatur_max': 300.35, 'date': '2022-08-12', 'waktu': '07:35:16', 'created': '2022-08-12T07:35:16.030311'}
{'kota': 'Lombok', 'cuaca': 'Clouds', 'cuaca_deskripsi': 'few clouds', 'temperatur': 298.36, 'temperatur_min': 298.36, 'temperatur_max': 298.36, 'date': '2022-08-12', 'waktu': '07:35:16', 'created': '2022-08-12T07:35:16.272541'}
{'kota': 'Labuan Bajo', 'cuaca': 'Clouds', 'cuaca_deskripsi': 'scattered clouds', 'temperatur': 301.19, 'temperatur_min': 301.19, 'temperatur_max': 301.19, 'date': '2022-08-12', 'waktu': '07:35:16', 'created': '2022-08-12T07:35:16.425575'}
{'kota': 'Medan', 'cuaca': 'Clouds', 'cuaca_deskripsi': 'scattered clouds', 'temperatur': 304.83, 'temperatur_min': 304.83, 'temperatur_max': 305.17, 'date': '2022-08-12', 'waktu': '07:35:16', 'created': '2022-08-12T07:35:16.585157'}
{'kota': 'Bandung', 'cuaca': 'Clouds', 'cuaca_deskripsi': 'broken c

{'kota': 'Labuan Bajo', 'cuaca': 'Clouds', 'cuaca_deskripsi': 'scattered clouds', 'temperatur': 301.19, 'temperatur_min': 301.19, 'temperatur_max': 301.19, 'date': '2022-08-12', 'waktu': '07:39:31', 'created': '2022-08-12T07:39:31.775949'}
{'kota': 'Medan', 'cuaca': 'Clouds', 'cuaca_deskripsi': 'broken clouds', 'temperatur': 304.52, 'temperatur_min': 304.52, 'temperatur_max': 304.52, 'date': '2022-08-12', 'waktu': '07:39:31', 'created': '2022-08-12T07:39:31.883754'}
{'kota': 'Bandung', 'cuaca': 'Rain', 'cuaca_deskripsi': 'light rain', 'temperatur': 300.11, 'temperatur_min': 300.11, 'temperatur_max': 300.11, 'date': '2022-08-12', 'waktu': '07:39:32', 'created': '2022-08-12T07:39:32.053583'}
{'kota': 'Bali', 'cuaca': 'Clouds', 'cuaca_deskripsi': 'broken clouds', 'temperatur': 300.35, 'temperatur_min': 300.35, 'temperatur_max': 300.35, 'date': '2022-08-12', 'waktu': '07:39:38', 'created': '2022-08-12T07:39:38.172712'}


{'kota': 'Lombok', 'cuaca': 'Clouds', 'cuaca_deskripsi': 'few clouds', 'temperatur': 298.36, 'temperatur_min': 298.36, 'temperatur_max': 298.36, 'date': '2022-08-12', 'waktu': '07:39:38', 'created': '2022-08-12T07:39:38.280173'}
{'kota': 'Labuan Bajo', 'cuaca': 'Clouds', 'cuaca_deskripsi': 'scattered clouds', 'temperatur': 301.19, 'temperatur_min': 301.19, 'temperatur_max': 301.19, 'date': '2022-08-12', 'waktu': '07:39:38', 'created': '2022-08-12T07:39:38.377908'}
{'kota': 'Medan', 'cuaca': 'Clouds', 'cuaca_deskripsi': 'scattered clouds', 'temperatur': 304.83, 'temperatur_min': 304.83, 'temperatur_max': 305.17, 'date': '2022-08-12', 'waktu': '07:39:38', 'created': '2022-08-12T07:39:38.496636'}
{'kota': 'Bandung', 'cuaca': 'Clouds', 'cuaca_deskripsi': 'broken clouds', 'temperatur': 300.11, 'temperatur_min': 300.11, 'temperatur_max': 300.11, 'date': '2022-08-12', 'waktu': '07:39:38', 'created': '2022-08-12T07:39:38.633644'}
